# QCD Stability Window Visualization

## Detailed Analysis

This script is used to generate a diagnostic plot that verifies the physical consistency of the generated Quark Star models. It visualizes the "Stability Window" in the microphysical parameter space—specifically the relationship between the Bag Constant ($B$), the Superconducting Gap ($\Delta$), and the Strange Quark Mass ($m_s$).

The plot serves two purposes:
1.  **Verification:** It confirms that the dataset generator (`worker_quark_gen.py`) correctly sampled parameters where strange quark matter is energetically stable relative to neutrons ($E/A < 939$ MeV).
2.  **Constraint Visualization:** It illustrates the theoretical "wedge" or window defined by the Bodmer-Witten hypothesis, bounded below by the stability of ordinary nuclei (Iron) and above by the stability of neutrons.

## Physics and Math

The stability of the Generalized Color-Flavor-Locked (CFL) phase is determined by the energy per baryon at zero pressure.

### Upper Bound (Neutron Stability)
For strange quark matter to be the true ground state of hadronic matter, its energy per baryon must be less than that of the neutron ($m_n = 939$ MeV). This imposes a maximum limit on the vacuum pressure $B$. The script calculates this limit as a function of $\Delta$ and $m_s$:

$$
B_{max}(\Delta, m_s) \approx \frac{1}{(\hbar c)^3} \left[ \frac{3}{4\pi^2}\mu_{lim}^4 + \frac{3}{\pi^2}\left(\Delta^2 - \frac{m_s^2}{4}\right)\mu_{lim}^2 \right]
$$

where $\mu_{lim} = m_n / 3$ is the critical chemical potential per quark.

### Lower Bound (Iron Stability)
Two-flavor quark matter (non-strange) must *not* be stable, otherwise ordinary atomic nuclei (like $^{56}\text{Fe}$) would spontaneously decay into quark matter. This imposes a minimum limit on $B$, represented in the code by the constant `Q_B_MIN` (typically derived from the condition $E/A > 930$ MeV for 2-flavor matter).

## Code Walkthrough

### 1. Data Filtering
The script selects only the Quark Star models (`Label == 1`) from the dataframe. Duplicate entries are removed based on `Curve_ID` to ensure that each point on the plot represents a unique Equation of State (EoS) parametrization.

```python
q_data = df[df['Label'] == 1].drop_duplicates(subset=['Curve_ID'])
```

### 2. Scatter Plot
The generated models are plotted as a scatter cloud.
*   **X-axis:** Gap Energy $\Delta$.
*   **Y-axis:** Bag Constant $B$.
*   **Color:** Strange Quark Mass $m_s$ (using a colormap). This reveals how $m_s$ influences the position of the models within the stable region.

### 3. Dynamic Stability Boundaries
Since the upper stability bound $B_{max}$ depends on $m_s$, there is no single "line" of stability. Instead, the script calculates a "Stability Uncertainty Band."

*   `b_max_ms_min`: Calculated using the minimum $m_s$ (e.g., 80 MeV).
*   `b_max_ms_max`: Calculated using the maximum $m_s$ (e.g., 120 MeV).

```python
# Calculate B_max vector for a specific ms
term1 = (3.0 / (4.0 * np.pi**2)) * (mu_limit**4)
eff_gap_sq = delta_arr**2 - (ms_val**2 / 4.0)
term2 = (3.0 / np.pi**2) * eff_gap_sq * (mu_limit**2)
return (term1 + term2) / (hc**3)
```

The region between these two curves is filled to represent the parameter space that *might* be stable depending on the specific value of $m_s$.

### 4. Shading Forbidden Regions
To clearly mark the physical limits, the script shades the invalid areas:
*   **Above the Band:** Unstable to Neutrons (matter would revert to hadronic state).
*   **Below the Iron Line:** Unstable to 2-flavor matter (ordinary matter would decay).

## Visualization Output

The script saves a single PDF file: `plots/fig_stability_triangle.pdf`.

*   **X-axis:** Gap Energy $\Delta$ [MeV].
*   **Y-axis:** Vacuum Pressure $B$ [MeV/fm$^3$].
*   **Visual Features:**
    *   A central "triangle" or wedge where the scattered data points reside.
    *   **Gray Shading:** Regions physically forbidden by QCD constraints.
    *   **Color Gradient:** Shows that for a fixed $B$ and $\Delta$, varying $m_s$ shifts the stability margin. Lower $m_s$ (darker colors) generally allows for slightly higher $B$.